<body>
    <center><h1 style="color:#FF0000";>Why do some Space Missions Fail?</h1></center>
    <center><h3> A Detailed Analysis on what factors contribute to failures </h3></center>
  <img src = https://worldwarwings.com/wp-content/uploads/2017/09/SpaceXplosions-735x413.jpg>
</body>

<div class="alert alert-block alert-info">
    <b>Note:</b> <p>Welcome to this notebook! Please be patient while the visualizations load.
</div>

# <center> ToC - What you can expect going forward</center>

- [An overview of the data](#data)
- [Analyzing how does the Launch Location Effect Space Missions](#lauloc)
- [What effect does the company undetaking the mission have on the Space Mission](#company)
- [Do Launch Vehicles have any impact?](#vehicle)
- [The most important factor - Time of Launch](#time)
- [Stonks or Stonksn't - Cost of Space Mission](#cost)
- [Modeling and Feature Importances](#models)

<a id = "data"></a>
# The Data

In [ ]:
df = pd.read_csv('../input/all-space-missions-from-1957/Space_Corrected.csv')
df.drop(['Unnamed: 0.1','Unnamed: 0'], axis = 1, inplace = True)
df.head()

The dataset has 7 Different Columns
- **Company Name** - The name of the company that undertook the space mission
- **Location** - The location of launch
- **Datum** - The date and time of the launch
- **Detail** - The Name of the Launch Vehicles
- **Status Rocket** - The current status of the rocket
- **Rocket** - The cost (in $ millions) of the mission
- **Status Mission** - The status, Success or Failure of the Launch

In [ ]:
#function to extract the name of the country from the location
def extract_country_name(location):
    country = location.split(',')[-1]
    country = country.strip()
    return country

#dictionary to help in mapping to get consistent and correct Country Names
countries_dict = {
    'Russia' : 'Russian Federation',
    'New Mexico' : 'USA',
    "Yellow Sea": 'China',
    "Shahrud Missile Test Site": "Iran",
    "Pacific Missile Range Facility": 'USA',
    "Barents Sea": 'Russian Federation',
    "Gran Canaria": 'USA'
}

In [ ]:
df['Country'] = df['Location'].apply(lambda x: extract_country_name(x))
df['Country'] = df['Country'].replace(countries_dict)

In [ ]:
#extracting date-time features
df['Datum'] = pd.to_datetime(df['Datum'])
df['year'] = df['Datum'].apply(lambda datetime: datetime.year)
df['month'] = df['Datum'].apply(lambda datetime: datetime.month)
df['weekday'] = df['Datum'].apply(lambda datetime: datetime.weekday())

In [ ]:
#function to get the Launch Vehicle Name from the Details
def getVehicles(detail):
    lv = []
    li = [x.strip() for x in detail.split('|')] #extracting the name of all launch vehicles from the Details section
    for ele in li:
        if('Cosmos' in ele):
            lv.append('Cosmos')
        elif('Vostok' in ele):
            lv.append('Vostok')
        elif('Tsyklon' in ele):
            lv.append('Tsyklon')
        elif('Ariane' in ele):
            lv.append('Ariane')
        elif('Atlas' in ele):
            lv.append('Atlas')
        elif('Soyuz' in ele):
            lv.append('Soyuz')
        elif('Delta' in ele):
            lv.append('Delta')
        elif('Titan' in ele):
            lv.append('Titan')
        elif('Molniya' in ele):
            lv.append('Molniya')
        elif('Zenit' in ele):
            lv.append('Zenit')
        elif('Falcon' in ele):
            lv.append('Falcon')
        elif('Long March' in ele):
            lv.append('Long March')
        elif('PSLV' in ele):
            lv.append('PSLV')
        elif('GSLV' in ele):
            lv.append('GSLV')
        elif('Thor' in ele):
            lv.append('Thor')
        else:
            lv.append('Other')
    return lv
df['Launch Vehicles'] = df['Detail'].apply(lambda x:getVehicles(x))

Since we are analyzing the reason why some space missions fail, let us first see the distribution of Status Mission. 
We can see that a large number(89.71%) of Space Missions are a Success and 7.84% of the Missions are a Failure. These 7.84% cases are the most important for our analysis.

In [ ]:
#creating a waffle Chart using pywaffle
plt.rcParams['figure.figsize'] = (7,12)
data = dict(df['Status Mission'].value_counts(normalize = True) * 100)
fig = plt.figure(
    FigureClass=Waffle, 
    columns=10, 
    values=data, 
    colors=("#3bff3b", "#ff3b3b", "#ffff3b","#ff9d3b"),
    title={'label': 'Status of Space Missions', 'loc': 'center'},
    icons = 'rocket',
    icon_size = 20,
    labels=[f"{k} ({v:.2f}%)" for k, v in data.items()],
    legend={'loc': 'lower left', 'bbox_to_anchor': (0, -0.4), 'ncol': len(data), 'framealpha': 0.3}
)
plt.show()

<a id = "lauloc"></a>
# Analyzing effect of Launch Location

In [ ]:
country_counts = dict(df['Country'].value_counts())
fig = go.Figure(data=[go.Table(
    header=dict(values=['<b>Country Name</b>', '<b>Number of Space Missions</b>'],
                line_color='black',
                fill_color='darkorange',
                align='left',
                font=dict(color='black', size=14)),
    cells=dict(values=[list(country_counts.keys()),
                      list(country_counts.values())],
               line_color='black',
               fill_color='white',
               align='left',
               font=dict(color='black', size=13)))
])

fig.update_layout(width=500, height=450,margin=dict(l=80, r=80, t=25, b=10),
                  title = { 'text' : '<b>Number of Space Missions Per Launch Location</b>', 'x' : 0.95},
                 font_family = 'Fira Code',title_font_color= '#ff0d00')
fig.show()

We see that a large number of Space Missions launch from Russia and USA. A lot of this has been because of the Space Race between the two countries.  
Here's an intro to the Space Race taken from [Wikipedia](https://en.wikipedia.org/wiki/Space_Race)
> The Space Race was a 20th-century competition between two Cold War rivals, the Soviet Union (USSR) and the United States (US), to achieve firsts in spaceflight capability. It had its origins in the ballistic missile-based nuclear arms race between the two nations following World War II. The technological advantage required to rapidly achieve spaceflight milestones was seen as necessary for national security, and mixed with the symbolism and ideology of the time. The Space Race led to pioneering efforts to launch artificial satellites, uncrewed space probes of the Moon, Venus, and Mars, and human spaceflight in low Earth orbit and to the Moon.

> The competition began in earnest on August 2, 1955, when the Soviet Union responded to the US announcement four days earlier of intent to launch artificial satellites for the International Geophysical Year, by declaring they would also launch a satellite "in the near future". The Soviet Union achieved the first successful launch with the October 4, 1957, orbiting of Sputnik 1, and sent the first human to space with the orbital flight of Yuri Gagarin on April 12, 1961. The USSR also sent the first woman, Valentina Tereshkova, to space on June 16, 1963, with numerous other firsts taking place over the next few years with regards to flight duration, spacewalks and related activities. According to Russian sources, these achievements led to the conclusion that the USSR had an advantage in space technology in the early 1960s.

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(df['Status Mission'])
colors = {0 : 'red', 1 : 'Orange', 2 : 'Yellow', 3 : 'Green'}

Now let us see the success and failure percentages of space missions for each of these countries going with the assumption that the Launch Location relates to the Country that was behind the Space Mission, that is to say that if the Launch Location is India, then we can think that all the ergonomics related to the mission were handled by India.

In [ ]:
fig = make_subplots(rows = 4 ,cols = 4,subplot_titles=df['Country'].unique())
for i, country in enumerate(df['Country'].unique()):
    counts = df[df['Country'] == country]['Status Mission'].value_counts(normalize = True) * 100
    color = [colors[x] for x in encoder.transform(counts.index)]
    trace = go.Bar(x = counts.index, y = counts.values, name = country,showlegend=False,marker={'color' : color})
    fig.add_trace(trace, row = (i//4)+1, col = (i%4)+1)
fig.update_layout(template = 'plotly_dark',margin=dict(l=80, r=80, t=50, b=10),
                  title = { 'text' : '<b>Countries and Mission Status</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#cacaca',height = 1000,width = 1100)
for i in range(1,5):
    fig.update_yaxes(title_text = 'Percentage',row = i, col = 1)
fig.show()

### We see that in terms of Success Percentages :
1. Kenya ranks first with a **100% success rate**. Kenya has made 9 Space Missions and all of them are Successful. 
2. France with 303 space missions comes in second with a Success percent of 94%
3. Russia with 1398 Space Missions comes at a close 3rd with a Success rate of 93.34%. Now that's something. In comparison to Launches taking place in USA, Russia fares better as US missions have a Success rate of around 88%

### In terms of Failure rate:  
1. Brazil and South Korea have a similar failure rate of 66.67%, i.e 2/3rd of their space missions fail. 
    * It should be noted that South Korea has a Success rate of 33%, while **Brazil is yet to make a successful Space mission.** 
    * These are not disheartening results,as both South Korea and Brazil have only made 3 Space Mission attempts.
2. Next, we have North Korea with a Failure rate of 60% in its 5 space missions.
3. Iran has a Failure rate of about 57%. It has however, undertaken only 14 space missions. 

In [ ]:
fig = px.sunburst(df,path = ['Status Mission','Country'])
fig.update_layout(margin=dict(l=80, r=80, t=25, b=10),
                  title = { 'text' : '<b>Countries and Mission Status</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#8000ff')
fig.show()

<a id = "company"></a>
# Analyzing effect of Company

In [ ]:
successPerc = df[df['Status Mission'] == 'Success'].groupby('Company Name')['Status Mission'].count()
for company in successPerc.index:
    successPerc[company] = (successPerc[company] / len(df[df['Company Name'] == company]))*100
successPerc = successPerc.sort_index()
FailurePerc = df[df['Status Mission'] == 'Failure'].groupby('Company Name')['Status Mission'].count()
for company in FailurePerc.index:
    FailurePerc[company] = (FailurePerc[company] / len(df[df['Company Name'] == company]))*100
FailurePerc = FailurePerc.sort_index()

In [ ]:
trace1 = go.Bar(x = successPerc.index, y = successPerc.values, name = 'Success Rate of Companies',opacity=0.7)
trace2 = go.Bar(x = FailurePerc.index, y = FailurePerc.values, name = 'Failure Rate of Companies',opacity=0.7)
fig = go.Figure([trace1,trace2])
fig.update_layout(template = 'plotly_white',margin=dict(l=80, r=80, t=25, b=10),
                  title = {'text' : '<b>Success and Failure Rates of Companies</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#8000ff',width = 1000,yaxis_title = '<b>Percentage</b>',xaxis_title = '<b>Companies</b>',
                 legend=dict(
                    yanchor="top",
                    y=0.99,
                    xanchor="left",
                    x=0.01
))

fig.show()

We measure the Success and Failure Rate of Each company. 

Success Rate highlights:
* Companies like ASI,Blue Origin,Douglas, IRGC,Khrunichev, OKB-586,Starsem, Yuzhmash and i-Space have a **100% success rate.** 

Failure Rate highlights
* Companies like EER, Landspace, OneSpace, Sandia and Virgin Orbit have a **100% failure rate**, i.e they haven't had any successful space mission. 

In [ ]:
fig = px.treemap(df,path = ['Status Mission','Country','Company Name'])
fig.update_layout(template = 'ggplot2',margin=dict(l=80, r=80, t=50, b=10),
                  title = { 'text' : '<b>Mission Status,Countries and Companies</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#ff6767')
fig.show()

### **Important Thing to Note**
From the above treemap, I was immediately interested in seeing **'RVSN USSR' having Launch Locations in both Kazakhstan and Russia.** 
After a bit of Googling and looking at the data, I found about the Baikonur Cosmodrome. Here's a piece from its wikipedia page:
> The Baikonur Cosmodrome is a spaceport located in an area of southern Kazakhstan leased to Russia.

>(...) **The spaceport is currently leased by the Kazakh Government to Russia until 2050, and is managed jointly by the Roscosmos State Corporation and the Russian Aerospace Forces.**

That ought to explain the reason why we use RVSN USSR entries in both Space Missions launched in Kazakhstan and Russia.

<a id = "vehicle"></a>
# Effect of Launch Vehicle

Here I am focusing on only a few prominent Launch Vehicles extracted from the Detail column of the dataset. We club together various versions of the Launch Vehicle like Cosmos 897, Cosmos 578 and Cosmos 1803 under the name of Cosmos. I am only taking into consideration 15 different Launch Vehicles and all the others are clubbed under the group of 'Others'

In [ ]:
# creating a single list containing the names of the Launch Vehicles
details = []
for detail in df.Detail.values:
    d = [x.strip() for x in detail.split('|')]
    for ele in d:
        if('Cosmos' in ele):
            details.append('Cosmos')
        elif('Vostok' in ele):
            details.append('Vostok')
        elif('Tsyklon' in ele):
            details.append('Tsyklon')
        elif('Ariane' in ele):
            details.append('Ariane')
        elif('Atlas' in ele):
            details.append('Atlas')
        elif('Soyuz' in ele):
            details.append('Soyuz')
        elif('Delta' in ele):
            details.append('Delta')
        elif('Titan' in ele):
            details.append('Titan')
        elif('Molniya' in ele):
            details.append('Molniya')
        elif('Zenit' in ele):
            details.append('Zenit')
        elif('Falcon' in ele):
            details.append('Falcon')
        elif('Long March' in ele):
            details.append('Long March')
        elif('PSLV' in ele):
            details.append('PSLV')
        elif('GSLV' in ele):
            details.append('GSLV')
        elif('Thor' in ele):
            details.append('Thor')
        else:
            details.append('Other')

In [ ]:
counts = dict(pd.Series(details).value_counts(sort = True))
fig = go.Figure(go.Bar(x = list(counts.keys()), y = list(counts.values())))
fig.update_layout(template = 'ggplot2',margin=dict(l=80, r=80, t=50, b=10),
                  title = { 'text' : '<b>Number of Missions in each type of Launch Vehicle</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#ff3434',
                 yaxis_title = '<b>Number of Missions</b>',xaxis_title = '<b>Launch Vehicle</b>',)
fig.show()

<a id = "time"></a>
# Analyzing the Effect of Launch Time

Now let us shift focus to analyszing the trend of failure in Space missions over time. 

In [ ]:
fig = make_subplots(rows = 3, cols = 1)
for i, period in enumerate(['year', 'month', 'weekday']):
    data = df[df['Status Mission'] == 'Failure'][period].value_counts().sort_index()
    data = dict((data / df[period].value_counts().sort_index())*100.0)
    mean = sum(data.values()) / len(data)
    if(period == 'year'):
        x = list(data.keys())
    elif(period == 'month'):
        x = ['January', 'February', 'March', 'April', 'May','June', 'July', 'August','September','October', 'November', 'December']
    else:
        x = ['Monday', 'Tuesday', 'Wednesday','Thursday','Friday','Saturday','Sunday']
    trace1 = go.Scatter(x = x, y = list(data.values()),mode = 'lines',text = list(data.keys()),name = f'Failures in each {period}',connectgaps = False)
    trace2 = go.Scatter(x = x, y = [mean]*len(data), mode = 'lines',showlegend=False,name = f'Mean failures over the {period}s',line = {'dash':'dash','color':
                                                                                                                                       'grey'})
    fig.append_trace(trace1, row = i+1, col = 1)
    fig.append_trace(trace2, row = i+1, col = 1)
fig.update_layout(template = 'simple_white',height = 600,
                  title = { 'text' : '<b>Failed Missions as a percentage of total missions in that period</b>', 'x' : 0.5})
for i in range(1,4):
    fig.update_yaxes(title_text = '<b>Percentage</b>',row = i, col = 1)
fig.show()

* **The overall percentage of failed missions has reduced over the years.** While in the early years we had failure rates as high as 60-70% the failure rate in 2018 is only around 1.7% and in 2019 it is about 5.5%. 
    * Clearly as the field of Space Exploration has evolved and as we have newer and more advanced technologies, the chances of failure have reduced. 
* **The mean failure rate over this time period is 9.6%. **
* Going over the monthly trends of failures, we see that **Space Missions Launched on November have the highest chance of failure followed by February.** 
* **December has the least failure rate of 5.78%**
* While weekly trend doesn't actually make sense (unless you are superstitious), but it was interesting to see that Wednesday could be termed as the safest day of the week in terms of Space Mission Launches.

<a id = "cost"></a>
# Analyzing the effect of Mission Cost

In [ ]:
df[' Rocket'] = df[' Rocket'].apply(lambda x: str(x).replace(',',''))
df[' Rocket'] = df[' Rocket'].astype('float64')
df[' Rocket'] = df[' Rocket'].fillna(0)

In [ ]:
costDict = dict(df[df[' Rocket'] > 0].groupby('year')[' Rocket'].mean())
fig = go.Figure(go.Scatter(x = list(costDict.keys()), y = list(costDict.values()), yaxis = 'y2',mode = 'lines',showlegend=False,name = 'Average Mission Cost Over the years'))
fig.update_layout(template = 'plotly_dark',margin=dict(l=80, r=80, t=50, b=10),
                  title = { 'text' : '<b>Average Mission Cost Over the years</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#cacaca',
                 yaxis_title = '<b>Cost of Mission in Million Dollars</b>',xaxis_title = '<b>Year of Launch</b>',)
fig.show()

We see that overtime, **the average mission cost has decreased** since 1987. However, as seen earlier, the failure rate of Space Missions has decreased over time. Thus, one thing in clear, **as the technology has advanced, we have been able to do space missions at a lower cost and with lower failure rates.** 

The reason why we see a very high spike in 1987 is because of a **Space Mission by RVSN USSR that has an estimated cost of 5000 million dollars.** Now, that's a lotta money! This skewed the data a lot as most other space missions of that year did not have any Mission Cost listed in the dataset. 

In [ ]:
fig = px.scatter(df[df[' Rocket'].between(1,4999)],x = 'year', y = 'Country', color = 'Status Mission',size = ' Rocket', size_max=30)
fig.update_layout(template = 'simple_white',margin=dict(l=80, r=80, t=50, b=10),
                  title = { 'text' : '<b>Average Mission Cost Over the years For Various Countries</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#00b300')
fig.show()

One important trend that we see is that the USA has managed to lower it's cost of Space Missions over time. 

In [ ]:
fig = px.scatter(df[df[' Rocket'].between(1,4999)],x = 'year', y = 'Company Name',color = 'Status Mission',size = ' Rocket',size_max = 30)
fig.update_layout(template = 'simple_white',margin=dict(l=80, r=80, t=50, b=10),
                  title = { 'text' : '<b>Average Mission Cost Over the years For Various Companies</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#00b300',height = 650)
fig.show()

The previous result of USA reducing the average mission cost over years is also confirmed here from the results shown from NASA. 

Another thing that I found particularly interesting is that the earlier SpaceX missions were unsuccessful and they had markedly low mission cost as compared to their later Space Missions. Thus, **increasing the budget that they allocate for each space mission helped them become more Successful.**

<a id = "models"></a>
# Finding Most Important Factors For Predicting Mission Status

In [ ]:
df['Target'] = (~(df['Status Mission'] == 'Success')).astype('int32')

In [ ]:
X = df[['Company Name',' Rocket','Country', 'year', 'month', 'weekday']]
encoder = LabelEncoder()
X.loc[:,'Company Name'] = encoder.fit_transform(X['Company Name'])
X.loc[:,'Country'] = encoder.fit_transform(X['Country'])
y = df['Target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0,stratify = y)

For the purpose of finding the most important features that could help us predict the mission status, we train a XGBoost Classifier model on the dataset.  

In [ ]:
classifier_xgb = xgb.XGBClassifier(random_state = 0, n_jobs = -1,max_depth = 5)
classifier_xgb.fit(X_train,y_train)

## Model Performance

Without any hyperparameter tuning, I train the model using the following features : 
- Company Name
- Location of Launch
- Rocket, i.e the cost of the mission
- Country 
- year of launch
- month of launch
- weekday of launch

The objective is to predict the target column, which in this case is the Status of the Mission. Care should be taken that in order to make things easier, I consider any mission that was not a Success as a failure,i.e, **partial failures and prelaunch failures are just failures in our eyes.**

### XGBoost

In [ ]:
print(f'Training Accuracy is {accuracy_score(y_train, classifier_xgb.predict(X_train))*100:.2f}%')
print(f'Testing Accuracy is {accuracy_score(y_test, classifier_xgb.predict(X_test))*100:.2f}%')

We train the classifier on the train set and make predictions on the test set which is 20% of the total data.

This simple XGBoost Model gives a baseline **accuracy of 93% on the train set and 89% on the test set.**

In [ ]:
feature_importance_xgb = classifier_xgb.get_booster().get_fscore()

In [ ]:
trace = go.Bar(x = list(feature_importance_xgb.values()), y = list(feature_importance_xgb.keys()),orientation='h')
fig = go.Figure([trace])
fig.update_layout(template = 'simple_white',margin=dict(l=80, r=80, t=50, b=10),
                  title = { 'text' : '<b>Feature Importance</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#00b300',
                 yaxis_title = '<b>Features</b>',xaxis_title = '<b>F Score</b>')

iplot(fig)

As seen from this feature importance graph, we see that the **Year in which the space mission was launched has the greatest impact in predicting the status of the mission.** Followed by the month in which the mission was launched. 

### RandomForest

In [ ]:
classifier_rf = RandomForestClassifier(random_state = 0, n_jobs = -1,max_depth = 5)
classifier_rf.fit(X_train,y_train)

In [ ]:
print(f'Training Accuracy is {accuracy_score(y_train, classifier_rf.predict(X_train))*100:.2f}%')
print(f'Testing Accuracy is {accuracy_score(y_test, classifier_rf.predict(X_test))*100:.2f}%')

Again with very little hyper-parameter tuning, I am able to get a **Training Accuracy of 90% and Testing Accuracy of 89%**

In [ ]:
trace = go.Bar(x = list(classifier_rf.feature_importances_), y = list(X_train.columns),orientation='h')
fig = go.Figure([trace])
fig.update_layout(template = 'simple_white',margin=dict(l=80, r=80, t=50, b=10),
                  title = { 'text' : '<b>Feature Importance</b>', 'x' : 0.5},
                 font_family = 'Fira Code',title_font_color= '#00b300',
                 yaxis_title = '<b>Features</b>',xaxis_title = '<b>Gini Importance</b>')

iplot(fig)

Again, even in the randomforest classifier, the most important feature is the year of mission launch followed by the Company name.

<div class="alert alert-block alert-info">
    <b>Thank You:</b> <p>In case you liked this kernel,please do upvote. Thank You again
</div>